In [ ]:
import imaplib
import email
from email.header import decode_header
import os
import pandas as pd

# credentials
username = 'your_email@gmail.com'
password = 'your_password'

# IMAP settings for gmail
imap_host = 'imap.gmail.com'
imap_port = 993

# Paths to csvs
csv_paths = {
    '1': '1.csv',
    '2': '2.csv',
    '3': '3.csv'
}


with open('password.txt', 'r') as file:
    subject_password = file.read().strip()


mail = imaplib.IMAP4_SSL(imap_host, imap_port)
mail.login(username, password)
mail.select("inbox")  # can change to another inbox if needed


# fetch emails with CSV attachments
def check_emails():
    status, messages = mail.search(None, '(UNSEEN)')
    if status == "OK":
        for num in messages[0].split():
            status, data = mail.fetch(num, '(RFC822)')
            if status == "OK":
                msg = email.message_from_bytes(data[0][1])
                subject = decode_header(msg["Subject"])[0][0]
                if isinstance(subject, bytes):
                    subject = subject.decode()  # decode if bytes
                
                if subject_password not in subject:
                    continue 

                target_file = None
                for key in csv_paths:
                    if key in subject:
                        target_file = csv_paths[key]
                        break
                
                if target_file is None:
                    continue 

                for part in msg.walk():
                    if part.get_content_maintype() == 'multipart':
                        continue
                    if part.get('Content-Disposition') is None:
                        continue
                    
                    filename = part.get_filename()
                    if filename and filename.endswith('.csv'):
                        filepath = os.path.join('/tmp', filename)
                        with open(filepath, 'wb') as f:
                            f.write(part.get_payload(decode=True))
                        
                        # Read the downlaod and append
                        new_data = pd.read_csv(filepath)
                        existing_data = pd.read_csv(target_file)
                        combined_data = pd.concat([existing_data, new_data])
                        combined_data.to_csv(target_file, index=False)
                        print(f"Appended data from {filename} to {target_file}.")

# Run the email check function
check_emails()

# Close the mail connection
mail.close()
mail.logout()
